# 强化学习实验
强化学习是一个理论与实践相结合的机器学习分支，我们不仅要理解它算法背后的一些数学原理，还要通过上机实践实现算法。在很多实验环境里面去探索算法能不能得到预期效果也是一个非常重要的过程。

我们可以使用 Python 和深度学习的一些包来实现强化学习算法。现在有很多深度学习的包可以使用，比如 PyTorch、TensorFlow、Keras，熟练使用其中的两三种，就可以实现非常多的功能。所以我们并不需要从头去“造轮子”。

## Gym
OpenAI是一家非营利性的人工智能研究公司，其公布了非常多的学习资源以及算法资源。其之所以叫作 OpenAI，他们把所有开发的算法都进行了开源。 如图 1.24 所示，OpenAI 的 Gym库是一个环境仿真库，里面包含很多现有的环境。针对不同的场景，我们可以选择不同的环境。离散控制场景（输出的动作是可数的，比如Pong游戏中输出的向上或向下动作）一般使用雅达利环境评估；连续控制场景（输出的动作是不可数的，比如机器人走路时不仅有方向，还有角度，角度就是不可数的，是一个连续的量 ）一般使用 MuJoCo 环境评估。Gym Retro是对 Gym 环境的进一步扩展，包含更多的游戏。

我们可以通过 pip 来安装 Gym 库，由于 Gym 库 0.26.0 及其之后的版本对之前的代码不兼容，所以我们安装 0.26.0 之前的 Gym，比如 0.25.2。
`pip install gym==0.25.2`
此外，为了显示图形界面，我们还需要安装 pygame 库。
`pip install pygame`

注意：建议大家在minicoda中创建一个专门的虚拟环境来运行这个项目

比如我们现在安装了Gym库，就可以直接调入Taxi-v3的环境。初始化这个环境后，我们就可以进行交互了。智能体得到某个观测后，它就会输出一个动作。这个动作会被环境拿去执行某个步骤 ，然后环境就会往前走一步，返回新的观测、奖励以及一个 flag 变量 done，done 决定这个游戏是不是结束了。我们通过几行代码就可实现强化学习的框架：

In [ ]:
import gym 
env = gym.make("Taxi-v3") 
observation = env.reset() 
agent = load_agent() 
for step in range(100):
    action = agent(observation) 
    observation, reward, done, info = env.step(action)

上面这段代码只是示例，其目的是让读者了解强化学习算法代码实现的框架，并非完整代码，load_agent 函数并未定义，所以运行这段代码会报错。

如图 1.25 所示，Gym 库里面有很多经典的控制类游戏。比如 Acrobot需要让一个双连杆机器人立起来；CartPole需要通过控制一辆小车，让杆立起来；MountainCar需要通过前后移动车，让它到达旗帜的位置。在刚开始测试强化学习的时候，我们可以选择这些简单环境，因为强化学习在这些环境中可以在一两分钟之内见到效果。
<div align="center">
<img src="https://datawhalechina.github.io/easy-rl/img/ch1/1.46.png" alt="经典控制问题" width="400" >
</div>
大家可以点[这个链接](https://www.gymlibrary.dev/environments/classic_control/)看一看这些环境。在刚开始测试强化学习的时候，可以选择这些简单环境，因为这些环境可以在一两分钟之内见到一个效果。

如图所示，CartPole-v0 环境有两个动作：将小车向左移动和将小车向右移动。我们还可以得到观测：小车当前的位置，小车当前往左、往右移的速度，杆的角度以及杆的最高点（顶端）的速度。 观测越详细，我们就可以更好地描述当前所有的状态。这里有奖励的定义，如果能多走一步，我们就会得到一个奖励（奖励值为1），所以我们需要存活尽可能多的时间来得到更多的奖励。当杆的角度大于某一个角度（没能保持平衡），或者小车的中心到达图形界面窗口的边缘，或者累积步数大于200，游戏就结束了，我们就输了。所以智能体的目的是控制杆，让它尽可能地保持平衡以及尽可能保持在环境的中央。
<div align="center">
<img src="https://datawhalechina.github.io/easy-rl/img/ch1/1.47.png" alt="经典控制问题" width="400" >
</div>

In [ ]:
import gym  # 导入 Gym 的 Python 接口环境包
env = gym.make('CartPole-v0')  # 构建实验环境
env.reset()  # 重置一个回合
for _ in range(1000):
    env.render()  # 显示图形界面
    action = env.action_space.sample() # 从动作空间中随机选取一个动作
    env.step(action) # 用于提交动作，括号内是具体的动作
env.close() # 关闭环境

注意：如果绘制了实验的图形界面窗口，那么关闭该窗口的最佳方式是调用 env.close()。试图直接关闭图形界面窗口可能会导致内存不能释放，甚至会导致死机。

当我们执行这段代码时，机器人会驾驶着小车朝某个方向一直跑，直到我们看不见，这是因为我们还没开始训练机器人。

Gym 库中的大部分小游戏都可以用一个普通的实数或者向量来表示动作。输出env.action_space.sample()的返回值，能看到输出为 1 或者 0。env.action_space.sample()的含义是，在该游戏的所有动作空间里随机选择一个作为输出。在这个例子中，动作只有两个：0 和 1，一左一右。env.step()方法有4个返回值：observation、reward、done、info 。

observation 是状态信息，是在游戏中观测到的屏幕像素值或者盘面状态描述信息。reward 是奖励值，即动作提交以后能够获得的奖励值。这个奖励值因游戏的不同而不同，但总体原则是，对完成游戏有帮助的动作会获得比较高的奖励值。done 表示游戏是否已经完成，如果完成了，就需要重置游戏并开始一个新的回合。info 是一些比较原始的用于诊断和调试的信息，或许对训练有帮助。不过，OpenAI 在评价我们提交的机器人时，是不允许使用这些信息的。

在每个训练中都要使用的返回值有 observation、reward、done。但 observation 的结构会由于游戏的不同而发生变化。以 CartPole-v0 为例，我们对代码进行修改：

In [ ]:
import gym  
env = gym.make('CartPole-v0')  
env.reset()  
for _ in range(1000):
    env.render()  
    action = env.action_space.sample() 
    observation, reward, done, info = env.step(action)
    print(observation)
env.close()    

从输出可以看出这是一个四维的观测。在其他游戏中会有维度更多的情况出现。

env.step()完成了一个完整的S→A→R→S过程。我们只要不断观测这样的过程，并让智能体在其中用相应的算法完成训练，就能得到一个高质量的强化学习模型。

Gym 库已注册的环境可以通过以下代码查看：

In [ ]:
from gym import envs
env_specs = envs.registry.all()
envs_ids = [env_spec.id for env_spec in env_specs]
print(envs_ids)

Gym 库中的每个环境都定义了观测空间和动作空间。观测空间和动作空间可以是离散的（取值为有限个离散的值），也可以是连续的（取值为连续的值）。

## MountainCar-v0 例子
接下来，我们通过一个例子来学习如何与 Gym 库进行交互。我们选取小车上山（MountainCar-v0）作为例子。

首先我们来看看这个任务的观测空间和动作空间：

In [ ]:
import gym
env = gym.make('MountainCar-v0')
print('观测空间 = {}'.format(env.observation_space))
print('动作空间 = {}'.format(env.action_space))
print('观测范围 = {} ~ {}'.format(env.observation_space.low,
        env.observation_space.high))
print('动作数 = {}'.format(env.action_space.n))

在 Gym 库中，环境的观测空间用 env.observation_space 表示，动作空间用 env.action_space 表示。离散空间 gym.spaces.Discrete 类表示，连续空间用 gym.spaces.Box 类表示。对于离散空间，Discrete (n) 表示可能取值的数量为 n；对于连续空间，Box类实例成员中的 low 和 high 表示每个浮点数的取值范围。MountainCar-v0 中的观测是长度为 2 的 numpy 数组，数组中值的类型为 float。MountainCar-v0 中的动作是整数，取值范围为 {0,1,2}。

接下来实现智能体来控制小车移动，对应代码如下。

In [ ]:
class SimpleAgent:
    def __init__(self, env):
        pass
    
    def decide(self, observation): # 决策
        position, velocity = observation
        lb = min(-0.09 * (position + 0.25) ** 2 + 0.03,
                0.3 * (position + 0.9) ** 4 - 0.008)
        ub = -0.07 * (position + 0.38) ** 2 + 0.07
        if lb < velocity < ub:
            action = 2
        else:
            action = 0
        return action # 返回动作

    def learn(self, *args): # 学习
        pass
    
agent = SimpleAgent(env)

SimpleAgent 类的 decide()方法用于决策，learn() 方法用于学习，该智能体不是强化学习智能体，不能学习，只能根据给定的数学表达式进行决策。

接下来我们试图让智能体与环境交互，代码如下。

In [ ]:
def play(env, agent, render=False, train=False):
    episode_reward = 0. # 记录回合总奖励，初始值为0
    observation = env.reset() # 重置游戏环境，开始新回合
    while True: # 不断循环，直到回合结束
        if render: # 判断是否显示
            env.render() # 显示图形界面
        action = agent.decide(observation)
        next_observation, reward, done, _ = env.step(action) # 执行动作
        episode_reward += reward # 收集回合奖励
        if train: # 判断是否训练智能体
            agent.learn(observation, action, reward, done) # 学习
        if done: # 回合结束，跳出循环
            break
        observation = next_observation
    return episode_reward # 返回回合总奖励

上面代码中的 play 函数可以让智能体和环境交互一个回合，该函数有 4 个参数。env 是环境类。agent 是智能体类。render 是 bool 型变量，其用于判断是否需要图形化显示。如果 render 为 True，则在交互过程中会调用 env.render() 以显示图形界面，通过调用 env.close() 可关闭图形界面。train 是 bool 型变量，其用于判断是否训练智能体，在训练过程中设置为 True，让智能体学习；在测试过程中设置为 False，让智能体保持不变。该函数的返回值 episode_reward 是 float 型的数值，其表示智能体与环境交互一个回合的回合总奖励。

接下来，我们使用下面的代码让智能体和环境交互一个回合，并显示图形界面。

In [ ]:
env.seed(3) # 设置随机种子，让结果可复现
episode_reward = play(env, agent, render=True)
print('回合奖励 = {}'.format(episode_reward))
env.close() # 关闭图形界面

为了评估智能体的性能，需要计算出连续交互 100 回合的平均回合奖励，代码如下。

In [ ]:
import numpy as np
episode_rewards = [play(env, agent) for _ in range(100)]
print('平均回合奖励 = {}'.format(np.mean(episode_rewards)))

SimpleAgent 类对应策略的平均回合奖励在 110 左右，而对于小车上山任务，只要连续 100 个回合的平均回合奖励大于110，就可以认为该任务被解决了。完整代码实现如下

测试智能体在 Gym 库中某个任务的性能时，出于习惯使然，学术界一般最关心 100 个回合的平均回合奖励。对于有些任务，还会指定一个参考的回合奖励值，当连续 100 个回合的奖励大于指定的值时，则认为该任务被解决了。而对于没有指定值的任务，就无所谓任务被解决了或没有被解决。

我们对 Gym 库的用法进行总结：使用 env=gym.make(环境名)取出环境，使用 env.reset()初始化环境，使用 env.step(动作)执行一步环境，使用 env.render()显示环境，使用 env.close()关闭环境。Gym库有对应的[官方文档](https://www.gymlibrary.dev/)，读者可以阅读文档来学习 Gym库 。

In [ ]:
import gym
import numpy as np


class SimpleAgent:
    def __init__(self, env):
        pass
    
    def decide(self, observation): # 决策
        position, velocity = observation
        lb = min(-0.09 * (position + 0.25) ** 2 + 0.03,
                0.3 * (position + 0.9) ** 4 - 0.008)
        ub = -0.07 * (position + 0.38) ** 2 + 0.07
        if lb < velocity < ub:
            action = 2
        else:
            action = 0
        return action # 返回动作

    def learn(self, *args): # 学习
        pass
    

def play(env, agent, render=False, train=False):
    episode_reward = 0. # 记录回合总奖励，初始化为0
    observation = env.reset() # 重置游戏环境，开始新回合
    while True: # 不断循环，直到回合结束
        if render: # 判断是否显示
            env.render() # 显示图形界面，图形界面可以用 env.close() 语句关闭
        action = agent.decide(observation)
        next_observation, reward, done, _ = env.step(action) # 执行动作
        episode_reward += reward # 收集回合奖励
        if train: # 判断是否训练智能体
            agent.learn(observation, action, reward, done) # 学习
        if done: # 回合结束，跳出循环
            break
        observation = next_observation
    return episode_reward # 返回回合总奖励


env = gym.make('MountainCar-v0')
env.seed(3) # 设置随机种子，让结果可复现
agent = SimpleAgent(env)
print('观测空间 = {}'.format(env.observation_space))
print('动作空间 = {}'.format(env.action_space))
print('观测范围 = {} ~ {}'.format(env.observation_space.low,
        env.observation_space.high))
print('动作数 = {}'.format(env.action_space.n))

episode_reward = play(env, agent, render=True)
print('回合奖励 = {}'.format(episode_reward))

episode_rewards = [play(env, agent) for _ in range(100)]
print('平均回合奖励 = {}'.format(np.mean(episode_rewards)))

## 用pytorch实现的深度强化学习
接下来，我们利用Tensorflow和Keras实现一个深度强化学习的项目，在强化学习中，我们需要理解`A-action`,`R-reward`,`E-environment`,`A-agent`这四个基本的概念；
接下来，我们会完成一下三件事情：

1，用Gym创建一个深度强化学习的环境

2，用pytorch创建一个深度学习模型

3，训练模型

4,可视化模型

### 安装环境
首先需要安装以下环境：

pip install pytorch(建议大家看官方文档安装安装库，以免出现不兼容的问题)


### 使用 OpenAI Gym 测试随机环境

In [ ]:
import gym 
import random

In [ ]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [ ]:
print(actions)
print(states)

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

env.close()

### 使用 pytorch 创建深度学习模型

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import numpy as np

In [ ]:
# 创建网络模型
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.linear1 = nn.Linear(input_dim, 128)
        self.linear2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x


In [ ]:
# 创建环境
env = gym.make('CartPole-v1',render_mode="human")
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n

In [ ]:

# 创建模型和优化器
model = DQN(input_dim, output_dim)
optimizer = optim.Adam(model.parameters())

In [ ]:

# 创建记忆
memory = deque(maxlen=10000)

In [ ]:
# 训练模型
for episode in range(15000):
    state = env.reset()
    total_reward = 0  # 初始化总奖励
    for t in range(200):
        action = env.action_space.sample() if random.random() < 0.1 else model(torch.from_numpy(state).float()).argmax().item()
        next_state, reward, done, _ = env.step(action)
        total_reward += reward  # 累加奖励
        memory.append((state, action, reward, next_state, done))
        state = next_state

        if done:
            break

    print(f"Episode {episode}, Total reward: {total_reward}")  # 输出这一轮的总奖励

    if len(memory) > 100:
        batch = random.sample(memory, 100)
        state, action, reward, next_state, done = zip(*batch)
        state = torch.from_numpy(np.array(state)).float()
        action = torch.tensor(action)
        reward = torch.tensor(reward)
        next_state = torch.from_numpy(np.array(next_state)).float()
        done = torch.tensor(done)

        q_values = model(state)
        next_q_values = model(next_state)

        q_value = q_values.gather(1, action.unsqueeze(1)).squeeze()
        next_q_value = next_q_values.max(1)[0]
        expected_q_value = reward + 0.99 * next_q_value * torch.logical_not(done)

        loss = (q_value - expected_q_value.detach()).pow(2).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
# 测试模型
total_reward = 0
state = env.reset()
for t in range(200):
    action = model(torch.from_numpy(state).float()).argmax().item()
    state, reward, done, _ = env.step(action)
    total_reward += reward
    env.render()
    if done:
        break
print("Total reward:", total_reward)

env.close()


In [ ]:
# 保存模型
torch.save(model.state_dict(), 'model.pth')

In [ ]:
# 加载模型
model = DQN(input_dim, output_dim)  # 替换为你的模型类
model.load_state_dict(torch.load('model.pth'))
model.eval()  # 设置为评估模式

# 测试模型
total_reward = 0
state = env.reset()
env.reset()
for t in range(200):
    # env.render()
    with torch.no_grad():  # 不需要计算梯度
        action = model(torch.from_numpy(state).float()).argmax().item()
    state, reward, done, _ = env.step(action)
    total_reward += reward
    if done:
        env.reset()

env.close()
print("Total reward:", total_reward)




## 介绍一下Gymnasium
Gymnasium是一个项目，为所有单智能体强化学习环境提供 API，并包括常见环境的实现：cartpole（倒立摆）、pendulum（摆锤）、mountain-car（山地车）、mujoco、atari 等等。

该 API 包含四个关键函数：make、reset、step 和 render。在 Gymnasium 的核心是 Env。

### 初始化环境
在 Gymnasium 中初始化环境非常简单，可以通过 make 函数完成：

首先，大家需要配置环境，请运行下面的代码；
```
pip install gymnasium
```

这还不够，还需要运行：(大家最好加上引号，如果你是Mac并且是zsh,那一定要加上引号)
```
pip install "gymnasium[atari]"
```

后面的库安装请大家自己学习一下，不会具体的说明了




In [ ]:
import gymnasium as gym
env = gym.make('CartPole-v1')

这将返回一个用于用户交互的 Env。要查看所有可创建的环境，请使用`gymnasium.envs.registry.keys()`。`make` 包括许多额外的参数，用于添加包装器、指定环境的关键字等等。

### 与环境进行交互

下图是强化学习的简化表示，Gymnasium 实现了这一过程。

<img src="https://gymnasium.farama.org/_images/AE_loop.png" alt="agent-environment loop" width="400" >

这个循环是使用以下 Gymnasium 代码实现的：



In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset()

for _ in range(1000):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

### 需要注意的地方
大家可能发现，运行上面的代码会出现报错；我们需要安装`swig`; SWIG (Simplified Wrapper and Interface Generator)是一个开源工具，用于连接 C/C++ 与其他高级编程语言（如 Python、Java、C# 等）之间的接口。

要安装SWIG，你可以按照以下步骤进行操作：

1. **与Python一起安装：**
   - 如果你使用的是Python，可以使用pip工具来安装SWIG。在命令行中输入以下命令：
     ```
     pip install swig
     ```
   这将会自动安装SWIG并将其与Python关联起来。

2. **作为一个独立的可执行文件安装：**
   - 如果你需要将SWIG作为一个独立的可执行文件安装，你可以从SWIG的官方网站下载适用于你的操作系统的安装包，并按照安装说明进行安装。
   - 请访问SWIG的官方网站（https://www.swig.org/download.html）下载适合你系统的安装程序。

无论你选择哪种方法，安装完成后，你就可以在命令行中使用SWIG了。你可以通过输入 `swig -version` 来验证SWIG是否成功安装，并查看其版本信息。

我建议第一种方法，如果大家对swig感兴趣，大家可以看看上面的官方网站；不过，大家装好了应该就可以用了；

大家可以很好完成下面的安装了
```
pip install 'gymnasium[box2d]'
```

ps:如果大家是windows，我建议大家多看看官方文档，需要学会配置swig,不然装了好像也没有用；


## 作业
请大家参考上面的代码，学习[gym官方文档](https://www.gymlibrary.dev/environments/classic_control/)，使用Mountain Car API ；完成利用深度学习方法加速汽车以达到右侧山顶的任务；
![alt text](https://www.gymlibrary.dev/_images/mountain_car.gif)

注意：
1，大家可以可能需要参考本文档之外的其他官方文档；
2，如果大家有问题可以随时联系助教【程骏，QQ：7879883】